In [1]:
f=open("fra.txt","r",encoding="utf-8").readlines()
en=[]
fre=[]
data=[]
for l in f:
    line=l.strip().split("\t")
    tmp={}
    tmp["en"]=line[0]
    tmp["fr"]=line[1]
    data.append(tmp)
print(len(data))
print(data[0])

170651
{'en': 'Go.', 'fr': 'Va !'}


In [2]:
# 它有3个列：input_ids、attention、labels，其中前两个列是tokenizer这个类的固有列，labels是预处理函数中加进去的一列
train_size=int(len(data)*0.9)
train_data=data[:train_size]
val_data=data[train_size:]

In [34]:
f=open("train.txt","w")
for i in train_data:
    f.write(str(i)+"\n")
f.close()

In [35]:
f=open("val.txt","w")
for i in val_data:
    f.write(str(i)+"\n")
f.close()

In [10]:
# 分词，使用已经训练好的helsinki-NLP/opus-mt-en-ro来做翻译任务
from transformers import AutoTokenizer
model_checkpoint="Helsinki-NLP/opus-mt-en-ro"
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint,use_fast=True)

/home/ma-user/work/anaconda3/envs/yzl_py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ma-user/work/anaconda3/envs/yzl_py37/lib/python3.7/site-packages/transformers/models/marian/tokenization_marian.py:196: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [11]:
tokenizer.source_lang="en_XX"
tokenizer.target_lang="fr_XX"

In [68]:
# 把以上内容合并成预处理函数
max_input_length=64
max_target_length=64
source_lang="en"
target_lang="fr"

def preprocess_function(examples):
    inputs=[eval(ex)[source_lang] for ex in examples["text"]]
    targets=[eval(ex)[target_lang] for ex in examples["text"]]
    model_inputs=tokenizer(inputs,max_length=max_input_length,truncation=True)
    # 为目标语言设置分词器
    with tokenizer.as_target_tokenizer():
        labels=tokenizer(targets,max_length=max_target_length,truncation=True)
    model_inputs["labels"]=labels["input_ids"]
    return model_inputs    

In [69]:
from datasets import load_dataset
raw_datasets=load_dataset("text",data_files={"train":"train.txt","validation":"val.txt"})

Using custom data configuration default-220cc82d5924ef47
Reusing dataset text (/home/ma-user/.cache/huggingface/datasets/text/default-220cc82d5924ef47/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)
100%|██████████| 2/2 [00:00<00:00, 1113.43it/s]


In [17]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 153585
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 17066
    })
})

In [70]:
tokenized_datasets=raw_datasets.map(preprocess_function,batched=True)

100%|██████████| 18/18 [00:02<00:00,  7.54ba/s]


In [86]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 153585
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 17066
    })
})

wandb: Network error (ProxyError), entering retry loop.
wandb: Network error (ProxyError), entering retry loop.
wandb: Network error (ProxyError), entering retry loop.
wandb: Network error (ProxyError), entering retry loop.
wandb: Network error (ProxyError), entering retry loop.


In [85]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 153585
    })
    validation: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 17066
    })
})

# 微调预训练模型

In [50]:
from transformers import AutoModelForSeq2SeqLM
model=AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [83]:
# 设定训练参数
from transformers import Seq2SeqTrainingArguments
batch_size=8
args=Seq2SeqTrainingArguments(
    "test-translation",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3, # 至多保存的模型个数
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [58]:
# 数据收集器data collator，告诉trainer如何从预处理的输入数据中构造batch，我使用数据处理器DataCollatorForSeq2Seq
# 将预处理的输入分batch再次处理后喂给模型
from transformers import DataCollatorForSeq2Seq
data_collator=DataCollatorForSeq2Seq(tokenizer,model=model)

In [61]:
# 定义评估方法，使用bleu指标，利用metric.compute计算该指标对模型进行评估
# 定义postprocess_text函数做一些数据后处理
import numpy as np

from datasets import load_metric
metric=load_metric("sacrebleu")
print("successfully import metric")

successfully import metric


In [80]:
def postprocess_text(preds,labels):
    preds=[pred.strip() for pred in preds]
    labels=[[label.strip()] for label in labels]
    return preds,labels

def compute_metrics(eval_preds):
    preds,labels=eval_preds
    if isinstance(preds,tuple):
        preds=preds[0]
    decoded_preds=tokenizer.batch_decode(preds,skip_special_tokens=True)

    # 如果labels=-100，说明这个label是无法编码的，应该用pad_token_id去进行填充
    labels=np.where(labels!=-100,labels,tokenizer.pad_token_id)
    decoded_labels=tokenizer.batch_decode(labels,skip_special_tokens=True)

    # 把预测后的输出转成适合metric.compute输入的格式
    print("type(decoded_preds)=",type(decoded_preds))
    print("type(decoded_labels)=",type(decoded_labels))
    decoded_preds,decoded_labels=postprocess_text(decoded_preds,decoded_labels)

    result=metric.compute(predictions=decoded_preds,references=decoded_labels)
    result={"bleu":result["score"]}
    
    prediction_lens=[np.count_nonzero(pred!=tokenizer.pad_token_id) for pred in preds]
    result["gen_len"]=np.mean(prediction_lens)
    result={k:round(v,4) for k,v in result.items()}
    print("result is as follow============================")
    print(result)
    return result


In [84]:
# 开始训练
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from transformers import Seq2SeqTrainer
trainer=Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"], # 这里要改成自己的训练集
    eval_dataset=tokenized_datasets["validation"], # 这里要改成自己的验证集
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: text. If text are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
/home/ma-user/work/anaconda3/envs/yzl_py37/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 153585
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 24000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.368700,0.797258,40.478500,28.469400
2,0.410000,0.748112,42.054000,28.373400
3,0.364800,0.722455,42.839800,28.343800
4,0.334500,0.710104,43.079300,28.455100
5,0.314800,0.706367,43.622100,28.369800
6,0.296300,0.701936,43.819400,28.322000
7,0.283000,0.699545,43.856200,28.431300
8,0.271700,0.698904,44.060500,28.350500
9,0.264600,0.698022,44.235000,28.312200
10,0.257000,0.699177,44.337100,28.340000


Saving model checkpoint to test-translation/checkpoint-500
Configuration saved in test-translation/checkpoint-500/config.json
Model weights saved in test-translation/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-translation/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-translation/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [test-translation/checkpoint-1000] due to args.save_total_limit
/home/ma-user/work/anaconda3/envs/yzl_py37/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to test-translation/checkpoint-1000
Configuration saved in test-translation/checkpoint-1000/config.json
Model weights saved in test-translation/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in tes

type(decoded_preds)= <class 'list'>
type(decoded_labels)= <class 'list'>
result is as follow============================
{'bleu': 40.4785, 'gen_len': 28.4694}


Saving model checkpoint to test-translation/checkpoint-2500
Configuration saved in test-translation/checkpoint-2500/config.json
Model weights saved in test-translation/checkpoint-2500/pytorch_model.bin
tokenizer config file saved in test-translation/checkpoint-2500/tokenizer_config.json
Special tokens file saved in test-translation/checkpoint-2500/special_tokens_map.json
Deleting older checkpoint [test-translation/checkpoint-1000] due to args.save_total_limit
/home/ma-user/work/anaconda3/envs/yzl_py37/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to test-translation/checkpoint-3000
Configuration saved in test-translation/checkpoint-3000/config.json
Model weights saved in test-translation/checkpoint-3000/pytorch_model.bin
tokenizer config file saved i

...

Saving model checkpoint to test-translation/checkpoint-22000
Configuration saved in test-translation/checkpoint-22000/config.json
Model weights saved in test-translation/checkpoint-22000/pytorch_model.bin
tokenizer config file saved in test-translation/checkpoint-22000/tokenizer_config.json
Special tokens file saved in test-translation/checkpoint-22000/special_tokens_map.json
Deleting older checkpoint [test-translation/checkpoint-20500] due to args.save_total_limit
/home/ma-user/work/anaconda3/envs/yzl_py37/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to test-translation/checkpoint-22500
Configuration saved in test-translation/checkpoint-22500/config.json
Model weights saved in test-translation/checkpoint-22500/pytorch_model.bin
tokenizer config fil

type(decoded_preds)= <class 'list'>
type(decoded_labels)= <class 'list'>
result is as follow============================
{'bleu': 44.3371, 'gen_len': 28.34}




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=24000, training_loss=0.3073067795435588, metrics={'train_runtime': 9196.9589, 'train_samples_per_second': 166.995, 'train_steps_per_second': 2.61, 'total_flos': 5799969455210496.0, 'train_loss': 0.3073067795435588, 'epoch': 10.0})

wandb: Network error (ReadTimeout), entering retry loop.
